# Preprocessing Phase


# Importing Libraries, Requirements and Datasets

In [1]:
#Installer tous les packages nécaissaires
!pip install -r ../requirements.txt

#ou bien

# !pip install autocorrect
# !pip install -U spacy
# !python -m spacy download fr_core_news_sm
# !pip install pyspellchecker
# !pip install keras==2.11.0
# !pip install xgboost
# !pip install h2o

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import openpyxl
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import nltk
import string                              # for string operations
from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer  
from nltk.corpus import stopwords
import spacy
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from collections import Counter

In [3]:

# Chemins des fichiers XLSX
file1 = '../resources/common/data/all_raw_comments_cleaning.xlsx'
file4 = '../resources/common/data/labled_comments.xlsx'

# Lecture des fichiers XLSX
dfnew_comment = pd.read_excel(file1)
df_labled = pd.read_excel(file4)

In [4]:
df_labled

,comment,score
0,aussi absorption directement niveau peau dit ...,Réclamation
1,doute plus pour efficacité produit,Réclamation
2,jai réclamé régler situation,Refus
3,jamais reçu échantillon auparavant,Refus
4,même sil observe antécédent chéloïde ème jour,Réclamation
...,...,...
2569,vient prescrire,Client
2570,vient prescrire moment visite,Client
2571,visite rappel,Présentation
2572,voi cest problème essentiellement gastrique,Réclamation


In [5]:
# dfnew_comment.drop('Unnamed: 1', axis=1, inplace=True)
dfnew_comment

,comment
0,Bon retour
1,Insistance
2,Insistance
3,Rappel
4,Elle a un bon retour sur produit
...,...
69893,Insistance
69894,Insistance
69895,Insistance
69896,Insistance


# Data Cleaning

In [6]:
def clean_text(text):
    # Vérifiez si le texte est une chaîne
    if isinstance(text, str):
        # Supprimer le texte de retweet de style ancien "RT"
        text = re.sub(r'^RT[\s]+', '', text)
        # Supprimer les hyperliens
        text = re.sub(r'https?://[^\s\n\r]+', '', text)
        # Supprimer les hashtags (seulement supprimer le signe de hash # du mot)
        text = re.sub(r'#', '', text)
        # Supprimer les dates au format AAAA-MM-JJ
        text = re.sub(r'\b\d{4}-\d{2}-\d{2}\b', '', text)
        # Supprimer l'heure au format HH:MM ou HH:MM:SS
        text = re.sub(r'\b\d{2}:\d{2}(:\d{2})?\b', '', text)
        # Supprimer les caractères spéciaux
        text = re.sub(r'[^\w\s]', '', text)
        # Supprimer les lignes vides ou les lignes avec juste un point
        text = re.sub(r'^(\s*\.?\s*)$', '', text, flags=re.MULTILINE)
        # Supprimer les accents
        text = unidecode(text)
    else:
        # Si le texte est un nombre, convertissez-le en chaîne de caractères
        if isinstance(text, (int, float)):
            text = str(text)
        # Si le texte est une valeur NaN, remplacez-le par une chaîne vide
        elif pd.isnull(text):
            text = ''
    return text.strip()  # Supprimer les espaces blancs en tête ou en queue
# Appliquer la fonction à chaque élément du DataFrame
dfnew_comment = dfnew_comment.applymap(clean_text)
# Remplacer les lignes qui sont juste un point ou une virgule (maintenant une chaîne vide après avoir supprimé les caractères spéciaux) par NaN
dfnew_comment.replace("", np.nan, inplace=True)
# Supprimer les lignes avec des valeurs NaN
dfnew_comment.dropna(subset=['comment'], inplace=True)



df_labled['comment'] = df_labled['comment'].apply(clean_text)

# Remplacer les lignes qui sont juste un point ou une virgule (maintenant une chaîne vide après avoir supprimé les caractères spéciaux) par NaN
df_labled.replace("", np.nan, inplace=True)

# Supprimer les lignes avec des valeurs NaN dans la colonne 'comment'
df_labled.dropna(subset=['comment'], inplace=True)


print(dfnew_comment)

                                comment
0                            Bon retour
1                            Insistance
2                            Insistance
3                                Rappel
4      Elle a un bon retour sur produit
...                                 ...
69893                        Insistance
69894                        Insistance
69895                        Insistance
69896                        Insistance
69897                      prescription

[69886 rows x 1 columns]


In [7]:
# Renommer dfnew_comment par df
df=dfnew_comment
print(df)
print(type(df))

                                comment
0                            Bon retour
1                            Insistance
2                            Insistance
3                                Rappel
4      Elle a un bon retour sur produit
...                                 ...
69893                        Insistance
69894                        Insistance
69895                        Insistance
69896                        Insistance
69897                      prescription

[69886 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>


# Tokenization 

In [8]:
print('\033[92m' + df)
print('\033[94m')

# instantiate tokenizer class
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)

# Tokeniser les textes dans la colonne 'comment'
df['tokens'] = df['comment'].apply(tokenizer.tokenize)
df_labled['tokens'] = df_labled['comment'].apply(tokenizer.tokenize)
# Afficher le DataFrame après tokenisation
print("\nDataFrame après tokenisation:")
print(df)




                                     comment
0                            Bon retour
1                            Insistance
2                            Insistance
3                                Rappel
4      Elle a un bon retour sur produit
...                                      ...
69893                        Insistance
69894                        Insistance
69895                        Insistance
69896                        Insistance
69897                      prescription

[69886 rows x 1 columns]


DataFrame après tokenisation:
                                comment  \
0                            Bon retour   
1                            Insistance   
2                            Insistance   
3                                Rappel   
4      Elle a un bon retour sur produit   
...                                 ...   
69893                        Insistance   
69894                        Insistance   
69895                        Insistance   
69896                 

In [9]:
print(df)

                                comment  \
0                            Bon retour   
1                            Insistance   
2                            Insistance   
3                                Rappel   
4      Elle a un bon retour sur produit   
...                                 ...   
69893                        Insistance   
69894                        Insistance   
69895                        Insistance   
69896                        Insistance   
69897                      prescription   

                                         tokens  
0                                 [bon, retour]  
1                                  [insistance]  
2                                  [insistance]  
3                                      [rappel]  
4      [elle, a, un, bon, retour, sur, produit]  
...                                         ...  
69893                              [insistance]  
69894                              [insistance]  
69895                            

# Importing French Stop Words

In [10]:
nltk.download('stopwords')
stopwords_french = stopwords.words('french')
print('Stop words\n')
print(stopwords_french)


Stop words

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'a

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\msi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
print('\033[94m')
df_clean1 = []
# Parcourir chaque liste de tokens
for tokens in df['tokens']:
    clean_tokens = []  # Liste pour stocker les tokens nettoyés d'un texte particulier
    for word in tokens:  # Parcourir chaque mot dans la liste de tokens
        # Vérifier si le mot n'est pas un mot d'arrêt et n'est pas un signe de ponctuation
        if word == 'pas' or word == 'ne' or (word not in stopwords_french and word not in string.punctuation):
            clean_tokens.append(word)
    # Ajouter les tokens nettoyés de ce texte à la liste df_clean
    df_clean1.append(clean_tokens)

# Vous pouvez maintenant ajouter df_clean comme une nouvelle colonne à votre DataFrame
df['clean_tokens'] = df_clean1
# Afficher le DataFrame
print(df)


                                comment  \
0                            Bon retour   
1                            Insistance   
2                            Insistance   
3                                Rappel   
4      Elle a un bon retour sur produit   
...                                 ...   
69893                        Insistance   
69894                        Insistance   
69895                        Insistance   
69896                        Insistance   
69897                      prescription   

                                         tokens               clean_tokens  
0                                 [bon, retour]              [bon, retour]  
1                                  [insistance]               [insistance]  
2                                  [insistance]               [insistance]  
3                                      [rappel]                   [rappel]  
4      [elle, a, un, bon, retour, sur, produit]  [a, bon, retour, produit]  
...                 

In [12]:
df_clean2 = []

for tokens in df_labled['tokens']:
    clean_tokens = []
    for word in tokens:
        if word == 'pas' or word == 'ne' or (word not in stopwords_french and word not in string.punctuation):
            clean_tokens.append(word)
    df_clean2.append(clean_tokens)

df_labled['clean_tokens'] = df_clean2

print(df_labled)

                                                comment         score  \
0     aussi absorption directement niveau peau dit p...   Réclamation   
1                    doute plus pour efficacite produit   Réclamation   
2                          jai reclame regler situation         Refus   
3                    jamais recu echantillon auparavant         Refus   
4         meme sil observe antecedent cheloide eme jour   Réclamation   
...                                                 ...           ...   
2569                                    vient prescrire        Client   
2570                      vient prescrire moment visite        Client   
2571                                      visite rappel  Présentation   
2572        voi cest probleme essentiellement gastrique   Réclamation   
2573  voit pas beaucoup  mai pour ancien oui peut pr...   Réclamation   

                                                 tokens  \
0     [aussi, absorption, directement, niveau, peau,...   
1    

In [13]:
# Créer une nouvelle colonne 'clean_data_noTokenized' en rejoignant les tokens nettoyés en une seule chaîne de caractères
df['clean_data_noTokenized'] = df['clean_tokens'].apply(' '.join)
df_labled['clean_data_noTokenized'] = df_labled['clean_tokens'].apply(' '.join)

# Afficher le DataFrame avec la nouvelle colonne 'clean_data_noTokenized'
print(df)

                                comment  \
0                            Bon retour   
1                            Insistance   
2                            Insistance   
3                                Rappel   
4      Elle a un bon retour sur produit   
...                                 ...   
69893                        Insistance   
69894                        Insistance   
69895                        Insistance   
69896                        Insistance   
69897                      prescription   

                                         tokens               clean_tokens  \
0                                 [bon, retour]              [bon, retour]   
1                                  [insistance]               [insistance]   
2                                  [insistance]               [insistance]   
3                                      [rappel]                   [rappel]   
4      [elle, a, un, bon, retour, sur, produit]  [a, bon, retour, produit]   
...            

# Lemmatizing

In [14]:
# Charger le modèle de langue français de spaCy
nlp = spacy.load('fr_core_news_sm')

# Créer une liste vide pour stocker les tokens lemmatisés
df_lemmatized1 = []

# Parcourir chaque liste de tokens
for tokens in df_clean1:
    lemmatized_tokens = []  # Liste pour stocker les tokens lemmatisés d'un texte particulier
    for word in tokens:  # Parcourir chaque mot dans la liste de tokens
        # Lemmatisation du mot
        doc = nlp(word)
        lemma = doc[0].lemma_ if doc else word
        lemmatized_tokens.append(lemma)  # Ajouter à la liste
    # Ajouter les tokens lemmatisés de ce texte à la liste df_lemmatized
    df_lemmatized1.append(lemmatized_tokens)
    
    
for i, text in enumerate(df_lemmatized1):
    text_str = ' '.join(text)
    text_str = re.sub(r'\bprescr\w*\b', 'prescrire', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\bsat\b|\bst\b|\bsatisfaire\b|\bsatisfaisant\b|\bsatisfaites\b', 'satisfait', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\brap(el|elle|ell)\b', 'rappel', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\bbc\b|\bbcp\b', 'beaucoup', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\baimer\b|\baime\b', 'aimer', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\btre\b', 'tres', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\bretou\b', 'retour', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\blindication\b', 'indication', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\bpresentationell\b', 'presentation', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\blutilis\b', 'utiliser', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\blachete\b', 'acheter', text_str, flags=re.IGNORECASE)

    # Remettre la chaîne de texte modifiée dans la liste df_lemmatized
    df_lemmatized1[i] = text_str.split()

# Vous pouvez maintenant ajouter df_lemmatized comme une nouvelle colonne à votre DataFrame
df['lemmatized_tokens'] = df_lemmatized1
# Afficher le DataFrame
print(df)

                                comment  \
0                            Bon retour   
1                            Insistance   
2                            Insistance   
3                                Rappel   
4      Elle a un bon retour sur produit   
...                                 ...   
69893                        Insistance   
69894                        Insistance   
69895                        Insistance   
69896                        Insistance   
69897                      prescription   

                                         tokens               clean_tokens  \
0                                 [bon, retour]              [bon, retour]   
1                                  [insistance]               [insistance]   
2                                  [insistance]               [insistance]   
3                                      [rappel]                   [rappel]   
4      [elle, a, un, bon, retour, sur, produit]  [a, bon, retour, produit]   
...            

In [15]:
# Charger le modèle de langue français de spaCy
nlp = spacy.load('fr_core_news_sm')

# Créer une liste vide pour stocker les tokens lemmatisés
df_lemmatized2 = []

# Parcourir chaque liste de tokens
for tokens in df_clean2:
    lemmatized_tokens = []  # Liste pour stocker les tokens lemmatisés d'un texte particulier
    for word in tokens:  # Parcourir chaque mot dans la liste de tokens
        # Lemmatisation du mot
        doc = nlp(word)
        lemma = doc[0].lemma_ if doc else word
        lemmatized_tokens.append(lemma)  # Ajouter à la liste
    # Ajouter les tokens lemmatisés de ce texte à la liste df_lemmatized
    df_lemmatized2.append(lemmatized_tokens)
    
    
for i, text in enumerate(df_lemmatized2):
    # Convertir la liste de tokens en une seule chaîne de texte
    text_str = ' '.join(text)
    text_str = re.sub(r'\bprescr\w*\b', 'prescrire', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\bsat\b|\bst\b|\bsatisfaire\b|\bsatisfaisant\b|\bsatisfaites\b', 'satisfait', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\brap(el|elle|ell)\b', 'rappel', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\bbc\b|\bbcp\b', 'beaucoup', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\baimer\b|\baime\b', 'aimer', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\btre\b', 'tres', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\bretou\b', 'retour', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\blindication\b', 'indication', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\bpresentationell\b', 'presentation', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\blutilis\b', 'utiliser', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'\blachete\b', 'acheter', text_str, flags=re.IGNORECASE)

    # Remettre la chaîne de texte modifiée dans la liste df_lemmatized
    df_lemmatized2[i] = text_str.split()

# Vous pouvez maintenant ajouter df_lemmatized comme une nouvelle colonne à votre DataFrame
df_labled['lemmatized_tokens'] = df_lemmatized2
# Afficher le DataFrame
print(df)

                                comment  \
0                            Bon retour   
1                            Insistance   
2                            Insistance   
3                                Rappel   
4      Elle a un bon retour sur produit   
...                                 ...   
69893                        Insistance   
69894                        Insistance   
69895                        Insistance   
69896                        Insistance   
69897                      prescription   

                                         tokens               clean_tokens  \
0                                 [bon, retour]              [bon, retour]   
1                                  [insistance]               [insistance]   
2                                  [insistance]               [insistance]   
3                                      [rappel]                   [rappel]   
4      [elle, a, un, bon, retour, sur, produit]  [a, bon, retour, produit]   
...            

# Most Frequent 50 Lemmatized Words

In [16]:
flat_list = [item for sublist in df_lemmatized1 for item in sublist]
# Compter les occurrences de chaque mot
word_freq = Counter(flat_list)
# Afficher les fréquences de mots
for word, count in word_freq.most_common(50):  # Affiche les 50 mots les plus fréquents
    print(f"{word}: {count}")

prescrire: 35480
rappel: 25293
produit: 13618
presentation: 6686
insistance: 4762
aller: 4586
entrain: 4337
exclusivite: 3853
bon: 3471
retour: 3410
avoir: 2255
etendue: 1952
surface: 1591
conseil: 1347
fois: 1008
quelque: 997
satisfait: 913
promesse: 784
demande: 780
representation: 762
disponible: 626
plaie: 617
gramme: 584
mentionner: 571
association: 516
place: 485
pas: 483
traitement: 463
patient: 457
cheloide: 440
relai: 421
mise: 401
tres: 388
stock: 372
surtout: 368
echantillon: 367
tout: 355
cas: 340
acte: 320
exclusivement: 301
engagement: 291
bien: 290
indication: 277
poste: 269
aussi: 249
post: 233
utilisation: 228
comme: 227
laser: 222
vaginal: 214


# Eliminer les mots dont leur frequence inferieure à 250

In [17]:
# # Flatten the list of lemmatized tokens
# flat_list = [item for sublist in df['lemmatized_tokens'] for item in sublist]
# # Calculate word frequencies
# word_freq = Counter(flat_list)
# # Display the least frequent words
# least_frequent_words = {word: count for word, count in word_freq.items() if count < 250} # you can choose a threshold that suits your data
# print("Least frequent words:", least_frequent_words)

In [18]:
# words_to_keep = {word for word, count in word_freq.items() if count >= 250}
# # Modify the lists of tokens in the DataFrame directly
# df['lemmatized_tokens'] = df['lemmatized_tokens'].apply(lambda tokens: [word for word in tokens if word in words_to_keep])
# # If you have another DataFrame (df_labled) that you want to apply the same transformation to:
# df_labled['lemmatized_tokens'] = df_labled['lemmatized_tokens'].apply(lambda tokens: [word for word in tokens if word in words_to_keep])

In [19]:
# Maintenant, recalculez les fréquences en utilisant la colonne mise à jour du DataFrame
# new_flat_list = [item for sublist in df['lemmatized_tokens'] for item in sublist]
# new_word_freq = Counter(new_flat_list)

# # Afficher les fréquences de mots après le filtrage
# for word, count in new_word_freq.most_common(50):  # Affiche les 50 mots les plus fréquents
#     print(f"{word}: {count}")


In [20]:
df_labled

,comment,score,tokens,clean_tokens,clean_data_noTokenized,lemmatized_tokens
0,aussi absorption directement niveau peau dit p...,Réclamation,"[aussi, absorption, directement, niveau, peau,...","[aussi, absorption, directement, niveau, peau,...",aussi absorption directement niveau peau dit p...,"[aussi, absorption, directement, niveau, peau,..."
1,doute plus pour efficacite produit,Réclamation,"[doute, plus, pour, efficacite, produit]","[doute, plus, efficacite, produit]",doute plus efficacite produit,"[doute, plus, efficacite, produit]"
2,jai reclame regler situation,Refus,"[jai, reclame, regler, situation]","[jai, reclame, regler, situation]",jai reclame regler situation,"[jai, reclame, regler, situation]"
3,jamais recu echantillon auparavant,Refus,"[jamais, recu, echantillon, auparavant]","[jamais, recu, echantillon, auparavant]",jamais recu echantillon auparavant,"[jamais, recu, echantillon, auparavant]"
4,meme sil observe antecedent cheloide eme jour,Réclamation,"[meme, sil, observe, antecedent, cheloide, eme...","[meme, sil, observe, antecedent, cheloide, eme...",meme sil observe antecedent cheloide eme jour,"[meme, sil, observer, antecedent, cheloide, em..."
...,...,...,...,...,...,...
2569,vient prescrire,Client,"[vient, prescrire]","[vient, prescrire]",vient prescrire,"[venir, prescrire]"
2570,vient prescrire moment visite,Client,"[vient, prescrire, moment, visite]","[vient, prescrire, moment, visite]",vient prescrire moment visite,"[venir, prescrire, moment, visite]"
2571,visite rappel,Présentation,"[visite, rappel]","[visite, rappel]",visite rappel,"[visite, rappel]"
2572,voi cest probleme essentiellement gastrique,Réclamation,"[voi, cest, probleme, essentiellement, gastrique]","[voi, cest, probleme, essentiellement, gastrique]",voi cest probleme essentiellement gastrique,"[voi, cest, probleme, essentiellement, gastrique]"


# Eliminer les 600 les moins mots corrélés avec les targets

In [21]:
import pandas as pd

# Lire le fichier mots.csv et créer un ensemble de mots à supprimer
words_to_remove = set()
with open('../resources/common/data/600_mots_moins_freq.csv', 'r') as file:
    for line in file:
        word = line.strip()  # supprime les espaces en début et en fin de ligne
        words_to_remove.add(word)

# Pour chaque ligne de df['lemmatized_tokens'], supprimez les mots présents dans mots.csv
def filter_tokens(tokens):
    return [token for token in tokens if token not in words_to_remove]

# Appliquer la fonction de filtrage à chaque ligne de lemmatized_tokens
df['lemmatized_tokens_finale'] = df['lemmatized_tokens'].apply(filter_tokens)
df_labled['lemmatized_tokens_finale'] = df_labled['lemmatized_tokens'].apply(filter_tokens)

# Afficher le DataFrame pour vérifier si les modifications ont été appliquées correctement
print(df)


                                comment  \
0                            Bon retour   
1                            Insistance   
2                            Insistance   
3                                Rappel   
4      Elle a un bon retour sur produit   
...                                 ...   
69893                        Insistance   
69894                        Insistance   
69895                        Insistance   
69896                        Insistance   
69897                      prescription   

                                         tokens               clean_tokens  \
0                                 [bon, retour]              [bon, retour]   
1                                  [insistance]               [insistance]   
2                                  [insistance]               [insistance]   
3                                      [rappel]                   [rappel]   
4      [elle, a, un, bon, retour, sur, produit]  [a, bon, retour, produit]   
...            

# Modeling Phase

In [22]:
df

,comment,tokens,clean_tokens,clean_data_noTokenized,lemmatized_tokens,lemmatized_tokens_finale
0,Bon retour,"[bon, retour]","[bon, retour]",bon retour,"[bon, retour]","[bon, retour]"
1,Insistance,[insistance],[insistance],insistance,[insistance],[insistance]
2,Insistance,[insistance],[insistance],insistance,[insistance],[insistance]
3,Rappel,[rappel],[rappel],rappel,[rappel],[rappel]
4,Elle a un bon retour sur produit,"[elle, a, un, bon, retour, sur, produit]","[a, bon, retour, produit]",a bon retour produit,"[avoir, bon, retour, produit]","[avoir, bon, retour, produit]"
...,...,...,...,...,...,...
69893,Insistance,[insistance],[insistance],insistance,[insistance],[insistance]
69894,Insistance,[insistance],[insistance],insistance,[insistance],[insistance]
69895,Insistance,[insistance],[insistance],insistance,[insistance],[insistance]
69896,Insistance,[insistance],[insistance],insistance,[insistance],[insistance]


In [23]:
df_labled

,comment,score,tokens,clean_tokens,clean_data_noTokenized,lemmatized_tokens,lemmatized_tokens_finale
0,aussi absorption directement niveau peau dit p...,Réclamation,"[aussi, absorption, directement, niveau, peau,...","[aussi, absorption, directement, niveau, peau,...",aussi absorption directement niveau peau dit p...,"[aussi, absorption, directement, niveau, peau,...","[absorption, directement, niveau, dire, pourqu..."
1,doute plus pour efficacite produit,Réclamation,"[doute, plus, pour, efficacite, produit]","[doute, plus, efficacite, produit]",doute plus efficacite produit,"[doute, plus, efficacite, produit]","[doute, plus, efficacite, produit]"
2,jai reclame regler situation,Refus,"[jai, reclame, regler, situation]","[jai, reclame, regler, situation]",jai reclame regler situation,"[jai, reclame, regler, situation]","[reclame, regler, situation]"
3,jamais recu echantillon auparavant,Refus,"[jamais, recu, echantillon, auparavant]","[jamais, recu, echantillon, auparavant]",jamais recu echantillon auparavant,"[jamais, recu, echantillon, auparavant]","[jamais, recu, echantillon, auparavant]"
4,meme sil observe antecedent cheloide eme jour,Réclamation,"[meme, sil, observe, antecedent, cheloide, eme...","[meme, sil, observe, antecedent, cheloide, eme...",meme sil observe antecedent cheloide eme jour,"[meme, sil, observer, antecedent, cheloide, em...","[meme, observer]"
...,...,...,...,...,...,...,...
2569,vient prescrire,Client,"[vient, prescrire]","[vient, prescrire]",vient prescrire,"[venir, prescrire]","[venir, prescrire]"
2570,vient prescrire moment visite,Client,"[vient, prescrire, moment, visite]","[vient, prescrire, moment, visite]",vient prescrire moment visite,"[venir, prescrire, moment, visite]","[venir, prescrire, moment, visite]"
2571,visite rappel,Présentation,"[visite, rappel]","[visite, rappel]",visite rappel,"[visite, rappel]","[visite, rappel]"
2572,voi cest probleme essentiellement gastrique,Réclamation,"[voi, cest, probleme, essentiellement, gastrique]","[voi, cest, probleme, essentiellement, gastrique]",voi cest probleme essentiellement gastrique,"[voi, cest, probleme, essentiellement, gastrique]","[voi, probleme, essentiellement, gastrique]"


In [24]:
# df.to_excel('../resources/dev_labo/data/processed/cleaned_data.xlsx', index=False)
# df_labled.to_excel ('../resources/dev_labo/data/processed/cleaned_labled_data.xlsx', index=False)


# fonction pour juger la qualité
def judge_quality(comment):
    if len(comment) < 5:
        return 'bad'
    else:
        return 'good'

# Création de la nouvelle DataFrame avec seulement les colonnes nécessaires
new_df = df[['comment', 'lemmatized_tokens']].copy()

# Création des nouvelles colonnes 'quality' et 'manual_clean_comments' (ici remplie avec des valeurs nulles pour l'instant)
# Appliquer la fonction à la colonne des commentaires
new_df['quality'] = new_df['comment'].apply(judge_quality)
new_df['manual_clean_comments'] = ''

# Réorganisation des colonnes dans l'ordre que vous voulez
new_df = new_df[['comment', 'quality', 'lemmatized_tokens', 'manual_clean_comments']]

# Enregistrement de la nouvelle DataFrame dans un fichier Excel
new_df.to_excel('../resources/dev_labo/data/processed/final_cleaned_Comments.xlsx', index=False)






In [25]:
# Charger les deux fichiers Excel dans des DataFrames
comments_classification_df = pd.read_excel('C:/Users/msi/Desktop/Stage_comments_classification/Comments-Classification-NLP/resources/dev_labo/data/processed/Comments_Classification.xlsx')
# Trouver les indices des lignes où 'manual_classification' est non nulle
indices = comments_classification_df[comments_classification_df['manual_classification'].notna()].index

# Pour chaque indice, copier la ligne correspondante dans labeled_df et la supprimer de comments_classification_df
for i in indices:
    row = comments_classification_df.loc[i]
    new_row = pd.DataFrame({'comment': [row['comment']], 'score': [row['manual_classification']]})
    df_labled = pd.concat([df_labled, new_row], ignore_index=True)
    comments_classification_df = comments_classification_df.drop(i)

# Sauvegarder les DataFrames mis à jour dans leurs fichiers Excel respectifs
# df_labled.to_excel('../resources/dev_labo/data/processed/cleaned_labled_data.xlsx', index=False)
comments_classification_df.to_excel('../resources/dev_labo/data/processed/Comments_Classification.xlsx', index=False)


# Semi-supervised Learning with LSTM

In [26]:
np.random.seed(42)

# Étape 1: Entraînez un modèle sur les données étiquetées

small_texts = df_labled['lemmatized_tokens_finale'].values
small_labels = df_labled['score'].values

# Encodage des étiquettes
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(small_labels)
small_y = to_categorical(encoded_labels)

# Tokenization et padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(small_texts)
sequences = tokenizer.texts_to_sequences(small_texts)
word_index = tokenizer.word_index
small_X = pad_sequences(sequences)

# Construction du modèle
model = Sequential()
model.add(Embedding(len(word_index) + 1, 128, input_length=small_X.shape[1]))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(small_y.shape[1], activation='softmax'))

# Compilation du modèle
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entraînement du modèle
model.fit(small_X, small_y, epochs=12, batch_size=32)

# Étape 2: Utilisez le modèle pour prédire des étiquettes pour les données non étiquetées

# Supposons que df soit votre DataFrame contenant les données non étiquetées
# et que la colonne 'lemmatized_tokens' contient les textes déjà nettoyés, tokenisés et lemmatisés

# Recombiner les tokens en texte
unlabeled_texts = [' '.join(tokens) for tokens in df['lemmatized_tokens_finale'].values]
unlabeled_sequences = tokenizer.texts_to_sequences(unlabeled_texts)
unlabeled_X = pad_sequences(unlabeled_sequences, maxlen=small_X.shape[1])

# Prédiction
predictions = model.predict(unlabeled_X)
predicted_labels = np.argmax(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_labels)

# Étape 3: Combinez les données étiquetées originales avec les données auto-étiquetées
combined_texts = np.concatenate((small_texts, unlabeled_texts))
combined_labels = np.concatenate((small_labels, predicted_labels))

# Encodage des étiquettes combinées
encoded_combined_labels = label_encoder.transform(combined_labels)
combined_y = to_categorical(encoded_combined_labels)

# Tokenization et padding des textes combinés
combined_sequences = tokenizer.texts_to_sequences(combined_texts)
combined_X = pad_sequences(combined_sequences, maxlen=small_X.shape[1])

# Étape 4: Ré-entraînez le modèle sur le nouvel ensemble de données combiné
model.fit(combined_X, combined_y, epochs=10, batch_size=32,shuffle=True)
# Préparer les données de la grande dataset pour la prédiction
# Comme précédemment mentionné, 'lemmatized_tokens' contient les textes déjà nettoyés, tokenisés et lemmatisés
large_unlabeled_texts = [' '.join(tokens) for tokens in df['lemmatized_tokens_finale'].values]
large_unlabeled_sequences = tokenizer.texts_to_sequences(large_unlabeled_texts)
large_unlabeled_X = pad_sequences(large_unlabeled_sequences, maxlen=small_X.shape[1])

# Utiliser le modèle pour prédire les étiquettes de la grande dataset
large_predictions = model.predict(large_unlabeled_X)

# Convertir les prédictions en étiquettes lisibles
large_predicted_labels = np.argmax(large_predictions, axis=1)
large_predicted_labels = label_encoder.inverse_transform(large_predicted_labels)

# Ajouter les étiquettes prédites à la grande dataset
df = df.copy()
df['predicted_score_lstm'] = large_predicted_labels

Epoch 1/12
81/81 [==============================] - 5s 18ms/step - loss: 1.7123 - accuracy: 0.3834
Epoch 2/12
81/81 [==============================] - 1s 18ms/step - loss: 1.1391 - accuracy: 0.6193
Epoch 3/12
81/81 [==============================] - 1s 16ms/step - loss: 0.8175 - accuracy: 0.7393
Epoch 4/12
81/81 [==============================] - 1s 17ms/step - loss: 0.6746 - accuracy: 0.7782
Epoch 5/12
81/81 [==============================] - 1s 17ms/step - loss: 0.5640 - accuracy: 0.8116
Epoch 6/12
81/81 [==============================] - 1s 16ms/step - loss: 0.5183 - accuracy: 0.8225
Epoch 7/12
81/81 [==============================] - 1s 16ms/step - loss: 0.4613 - accuracy: 0.8427
Epoch 8/12
81/81 [==============================] - 1s 16ms/step - loss: 0.4443 - accuracy: 0.8430
Epoch 9/12
81/81 [==============================] - 1s 16ms/step - loss: 0.4145 - accuracy: 0.8609
Epoch 10/12
81/81 [==============================] - 1s 16ms/step - loss: 0.3953 - accuracy: 0.8640
Epoch 11/

In [27]:
df['predicted_score_lstm'].unique()

array(['Partenaire', 'Présentation', 'Client', 'promesse', 'Réclamation',
       'test en cours', 'Refus', 'Livraison'], dtype=object)

# Semi-supervised Learning with TfidfVectorizer and Naive Bayes

In [28]:
# # Préparation des données étiquetées
# df_labled = df_labled.copy()
# df_labled['lemmatized_tokens_finale'] = df_labled['lemmatized_tokens_finale'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
# small_texts = df_labled['lemmatized_tokens_finale'].values
# small_labels = df_labled['score'].values

# # Création d'un modèle - pipeline TF-IDF suivi d'un Naive Bayes
# model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# # Entraînement du modèle avec les données étiquetées
# model.fit(small_texts, small_labels)

# # Prédire sur la grande dataset
# unlabeled_texts = [' '.join(tokens) for tokens in df['lemmatized_tokens_finale'].values]

# # Utiliser le modèle pour prédire les étiquettes de la grande dataset
# large_predicted_labels = model.predict(unlabeled_texts)

# # Ajouter les étiquettes prédites à la grande dataset
# df = df.copy()
# df['predicted_score_TfidfVectorizer'] = large_predicted_labels


# Semi-supervised Learning with XGBoost


In [29]:
# import numpy as np
# import xgboost as xgb
# from gensim.models import Word2Vec
# from sklearn import preprocessing
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report

# # Créer des embeddings de mots avec Word2Vec
# sentences = [text.split() for text in df_labled['lemmatized_tokens_finale'].values]
# word2vec_model = Word2Vec(sentences, vector_size=300, window=5, min_count=1, workers=4)
# word2vec_model.save("xgbmodel/word2vec.model")

# # Convertir les textes en représentations vectorielles en utilisant les embeddings de mots
# def text_to_vector(text, model):
#     words = text.split()
#     vector = np.zeros(model.vector_size)
#     for word in words:
#         if word in model.wv:
#             vector += model.wv[word]
#     return vector / (len(words) + 1e-5)

# X = np.array([text_to_vector(text, word2vec_model) for text in df_labled['lemmatized_tokens_finale'].values])
# small_labels = df_labled['score'].values

# # Encodez les étiquettes en valeurs numériques
# label_encoder = preprocessing.LabelEncoder()
# encoded_labels = label_encoder.fit_transform(small_labels)

# # Diviser les données en ensembles d'entraînement et de test
# X_train, X_test, y_train, y_test = train_test_split(X, encoded_labels, test_size=0.2, random_state=42)

# # Entraîner un modèle XGBoost
# params = {
#     'objective': 'multi:softmax',
#     'num_class': len(label_encoder.classes_),
#     'max_depth': 8,  # Augmentation de la profondeur
#     'eta': 0.1,  # Augmentation du taux d'apprentissage
#     'subsample': 0.8,
#     'colsample_bytree': 0.8,  # Ajout d'un hyperparamètre
#     'min_child_weight': 1  # Ajout d'un hyperparamètre
# }
# dtrain = xgb.DMatrix(X_train, label=y_train)
# dtest = xgb.DMatrix(X_test, label=y_test)
# bst = xgb.train(params, dtrain, num_boost_round=200, early_stopping_rounds=10, evals=[(dtest, 'eval')])

# # Évaluer les performances sur l'ensemble de test
# y_pred = bst.predict(dtest)
# print(classification_report(y_test, y_pred))

# # Préparer les données non étiquetées
# X_unlabeled = np.array([text_to_vector(' '.join(tokens), word2vec_model) for tokens in df['lemmatized_tokens_finale'].values])

# # Utiliser le modèle pour prédire les étiquettes de la grande dataset
# d_unlabeled = xgb.DMatrix(X_unlabeled)
# large_predictions = bst.predict(d_unlabeled)

# # Convertir les prédictions en étiquettes lisibles
# large_predicted_labels = label_encoder.inverse_transform(large_predictions.astype(int))

# # Create a copy of the DataFrame to avoid SettingWithCopyWarning
# df = df.copy()

# # Add the predicted labels to the DataFrame
# df['predicted_score_XGB'] = large_predicted_labels



In [30]:
# from sklearn.tree import DecisionTreeClassifier

# # Étape 1: Entraînez un modèle sur les données étiquetées
# small_texts = df_labled['lemmatized_tokens_finale'].values
# small_labels = df_labled['score'].values

# # Encodage des étiquettes
# label_encoder = LabelEncoder()
# encoded_labels = label_encoder.fit_transform(small_labels)

# # Construction du modèle d'arbre de décision
# tree_model = DecisionTreeClassifier()

# # Entraînement du modèle
# tree_model.fit(small_X, encoded_labels)

# # Étape 2: Utilisez le modèle pour prédire des étiquettes pour les données non étiquetées
# unlabeled_texts = [' '.join(tokens) for tokens in df['lemmatized_tokens_finale'].values]
# unlabeled_sequences = tokenizer.texts_to_sequences(unlabeled_texts)
# unlabeled_X = pad_sequences(unlabeled_sequences, maxlen=small_X.shape[1])

# # Prédiction
# predicted_labels = tree_model.predict(unlabeled_X)
# predicted_labels = label_encoder.inverse_transform(predicted_labels)

# # Étape 3: Combinez les données étiquetées originales avec les données auto-étiquetées
# combined_texts = np.concatenate((small_texts, unlabeled_texts))
# combined_labels = np.concatenate((small_labels, predicted_labels))

# # Encodage des étiquettes combinées
# encoded_combined_labels = label_encoder.transform(combined_labels)

# # Étape 4: Ré-entraînez le modèle sur le nouvel ensemble de données combiné
# tree_model.fit(combined_X, encoded_combined_labels)

# # Prédiction sur la grande dataset
# large_unlabeled_texts = [' '.join(tokens) for tokens in df['lemmatized_tokens_finale'].values]
# large_unlabeled_sequences = tokenizer.texts_to_sequences(large_unlabeled_texts)
# large_unlabeled_X = pad_sequences(large_unlabeled_sequences, maxlen=small_X.shape[1])

# # Utilisation du modèle pour prédire les étiquettes de la grande dataset
# large_predicted_labels = tree_model.predict(large_unlabeled_X)
# large_predicted_labels = label_encoder.inverse_transform(large_predicted_labels)

# # Ajouter les étiquettes prédites à la grande dataset
# df['predicted_score_decision_tree'] = large_predicted_labels


In [31]:
# from sklearn.ensemble import RandomForestClassifier

# # Construction du modèle de forêt aléatoire
# rf_model = RandomForestClassifier()

# # Entraînement du modèle
# rf_model.fit(small_X, encoded_labels)

# # Prédiction sur les données non étiquetées
# predicted_labels = rf_model.predict(unlabeled_X)
# predicted_labels = label_encoder.inverse_transform(predicted_labels)

# # Ré-entraînement du modèle sur l'ensemble de données combiné
# rf_model.fit(combined_X, encoded_combined_labels)

# # Prédiction sur la grande dataset
# large_predicted_labels = rf_model.predict(large_unlabeled_X)
# large_predicted_labels = label_encoder.inverse_transform(large_predicted_labels)

# # Ajouter les étiquettes prédites à la grande dataset
# df['predicted_score_random_forest'] = large_predicted_labels


In [32]:
df

,comment,tokens,clean_tokens,clean_data_noTokenized,lemmatized_tokens,lemmatized_tokens_finale,predicted_score_lstm
0,Bon retour,"[bon, retour]","[bon, retour]",bon retour,"[bon, retour]","[bon, retour]",Partenaire
1,Insistance,[insistance],[insistance],insistance,[insistance],[insistance],Présentation
2,Insistance,[insistance],[insistance],insistance,[insistance],[insistance],Présentation
3,Rappel,[rappel],[rappel],rappel,[rappel],[rappel],Présentation
4,Elle a un bon retour sur produit,"[elle, a, un, bon, retour, sur, produit]","[a, bon, retour, produit]",a bon retour produit,"[avoir, bon, retour, produit]","[avoir, bon, retour, produit]",Partenaire
...,...,...,...,...,...,...,...
69893,Insistance,[insistance],[insistance],insistance,[insistance],[insistance],Présentation
69894,Insistance,[insistance],[insistance],insistance,[insistance],[insistance],Présentation
69895,Insistance,[insistance],[insistance],insistance,[insistance],[insistance],Présentation
69896,Insistance,[insistance],[insistance],insistance,[insistance],[insistance],Présentation


In [33]:
import pandas as pd

# Supposons que 'df' est votre DataFrame d'origine.
# Extraction des colonnes nécessaires
extracted_df = df[['comment', 'predicted_score_lstm']].copy()

# Ajout de la nouvelle colonne 'manual_classification' (ici remplie avec des valeurs nulles pour l'instant)
extracted_df['manual_classification'] = None

# Exporter le DataFrame extrait vers un fichier Excel
extracted_df.to_excel('../resources/dev_labo/data/processed/Comments_Classification.xlsx', index=False)


In [41]:
import pandas as pd

# lire les deux fichiers Excel dans des DataFrame pandas
df1 = pd.read_excel("../resources/dev_labo/data/processed/Comments_Classification.xlsx")
df2 = pd.read_excel("../resources/common/data/labled_comments.xlsx")

# renommer la colonne `score` de df2 en `manual_classification`
df2.rename(columns={'score': 'manual_classification'}, inplace=True)

# ajouter une colonne 'predicted_score_lstm' vide à df2 pour correspondre aux colonnes de df1
df2['predicted_score_lstm'] = None

# aligner les colonnes de df2 avec celles de df1
df2 = df2[df1.columns]

# fusionner les deux DataFrame
merged_df = pd.concat([df1, df2], ignore_index=True)

# écrire le DataFrame fusionné dans un nouveau fichier Excel
merged_df.to_excel("../resources/dev_labo/data/processed/merged_comments.xlsx", index=False)
